# Määrämuotoisen datan käsittelyä pythonilla (excel/csv)

Muiden nykyaikaisten ohjelmointikielien tapaan Python sisältää hyvin paljon valmiina erilaisia apukirjastoja, joita yhdistelemällä saa aikaiseksi tehokkaita toimintoja.

Otamme esimerksi tilanteen, jossa euroopan keskuspankista ladattu euron valuuttakurssidata halutaan saada paremmin jatkokäsittelyyn sopivaan muotoon. Sivulla https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/index.en.html löytyy data csv-muotoisena. Tässä zip-pakatussa tiedostossa on data sarakkeissa - me haluaisimme saadan datan rivi kerrallaan ja siten, että kolmikirjaimiset maakoodit ovat aakkosjärjestyksessä.

Teemme datan lataamisen ja muokkaamisen käyttänen python3.5:n standardikirjastoja.

In [1]:
#!/usr/bin/env python3

# otetaan käyttöön tarvittavista moduuleista olioita ja funktioita
from io      import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# ladataan datana (bytes) URL:n sisältö
# tässä ei ole virhetarkastelua - ongelmatilanteissa saamme poikkeuksen 
urldata = urlopen('https://www.ecb.europa.eu/stats/eurofxref/eurofxref.zip').read()

print( 'Datatype', type(urldata), 'size', len(urldata) )

Datatype <class 'bytes'> size 403


In [2]:
# verkosta tulleet raakatavut muutetaan olioksi, jota voi käsitellä kuten tiedostoa
bytefile = BytesIO( urldata )

# tämä nyt muistissa oleva "tiedosto" avataan käsiteltäväksi zipfile-moduulin avulla
zipfile = ZipFile( bytefile )
print( zipfile.filelist ) # zip-paketin sisältö

[<ZipInfo filename='eurofxref.csv' compress_type=deflate file_size=434 compress_size=279>]


In [3]:
# varmistetaan oletuksemme ladatusta zip-paketista:
assert( len( zipfile.filelist ) == 1 ) # sisällä yksi tiedosto
fileobj = zipfile.filelist[0] 
assert( fileobj.filename == 'eurofxref.csv' ) # tiedostolla on oletettu nimi
print( 'checks ok ')

checks ok 


In [4]:
# kerätään pakatun zip-tiedoston sisältö talteen listaksi rivejä
with zipfile.open( fileobj.filename ) as infile:
    LINES = infile.readlines()
    
print( LINES )

[b'Date, USD, JPY, BGN, CZK, DKK, GBP, HUF, PLN, RON, SEK, CHF, NOK, HRK, RUB, TRY, AUD, BRL, CAD, CNY, HKD, IDR, INR, KRW, MXN, MYR, NZD, PHP, SGD, THB, ZAR, ILS, \n', b'08 March 2017, 1.0556, 120.65, 1.9558, 27.021, 7.4335, 0.86753, 310.70, 4.3130, 4.5503, 9.5223, 1.0702, 8.9513, 7.4213, 61.7573, 3.9434, 1.3968, 3.3151, 1.4193, 7.2953, 8.1982, 14144.80, 70.4960, 1216.36, 20.7130, 4.7082, 1.5219, 53.198, 1.4947, 37.236, 13.7908, 3.8879, \n']


In [5]:
# jokaisella rivillä on pilkulla eroteltuna kiinnostava data,
# joten pätkitään data listaksi pilkun kohdalta.
# tyyppimuunnos merkkijonoksi tuottaa pilkkomisen jälkeen myös merkkijonoja
HEADER = str( LINES[0] ).split(', ')
DATA   = str( LINES[1] ).split(', ')

print( HEADER, DATA )

["b'Date", 'USD', 'JPY', 'BGN', 'CZK', 'DKK', 'GBP', 'HUF', 'PLN', 'RON', 'SEK', 'CHF', 'NOK', 'HRK', 'RUB', 'TRY', 'AUD', 'BRL', 'CAD', 'CNY', 'HKD', 'IDR', 'INR', 'KRW', 'MXN', 'MYR', 'NZD', 'PHP', 'SGD', 'THB', 'ZAR', 'ILS', "\\n'"] ["b'08 March 2017", '1.0556', '120.65', '1.9558', '27.021', '7.4335', '0.86753', '310.70', '4.3130', '4.5503', '9.5223', '1.0702', '8.9513', '7.4213', '61.7573', '3.9434', '1.3968', '3.3151', '1.4193', '7.2953', '8.1982', '14144.80', '70.4960', '1216.36', '20.7130', '4.7082', '1.5219', '53.198', '1.4947', '37.236', '13.7908', '3.8879', "\\n'"]


In [6]:
# ensimmäinen ja viimeinen alkio ei kiinnosta meitä, joten pätkäistään ne pois listalta
HEADER = HEADER[1:-1]
DATA   = DATA[1:-1]

print( HEADER, DATA )

['USD', 'JPY', 'BGN', 'CZK', 'DKK', 'GBP', 'HUF', 'PLN', 'RON', 'SEK', 'CHF', 'NOK', 'HRK', 'RUB', 'TRY', 'AUD', 'BRL', 'CAD', 'CNY', 'HKD', 'IDR', 'INR', 'KRW', 'MXN', 'MYR', 'NZD', 'PHP', 'SGD', 'THB', 'ZAR', 'ILS'] ['1.0556', '120.65', '1.9558', '27.021', '7.4335', '0.86753', '310.70', '4.3130', '4.5503', '9.5223', '1.0702', '8.9513', '7.4213', '61.7573', '3.9434', '1.3968', '3.3151', '1.4193', '7.2953', '8.1982', '14144.80', '70.4960', '1216.36', '20.7130', '4.7082', '1.5219', '53.198', '1.4947', '37.236', '13.7908', '3.8879']


In [7]:
# data otetaan käsittelyyn liukulukuina
DATA = list( map( float, DATA ) )

print( DATA )

[1.0556, 120.65, 1.9558, 27.021, 7.4335, 0.86753, 310.7, 4.313, 4.5503, 9.5223, 1.0702, 8.9513, 7.4213, 61.7573, 3.9434, 1.3968, 3.3151, 1.4193, 7.2953, 8.1982, 14144.8, 70.496, 1216.36, 20.713, 4.7082, 1.5219, 53.198, 1.4947, 37.236, 13.7908, 3.8879]


In [8]:
# rakennetaan DICTIONARY, jossa on jokaisen maatunnuksen avaimella talletettuna valuuttakurssi arvona
RATES = dict()
for country, rate in zip( HEADER, DATA ):
    RATES[ country ] = rate
    
print( RATES )

{'CAD': 1.4193, 'BGN': 1.9558, 'GBP': 0.86753, 'BRL': 3.3151, 'CNY': 7.2953, 'RUB': 61.7573, 'HKD': 8.1982, 'CHF': 1.0702, 'HUF': 310.7, 'NOK': 8.9513, 'SGD': 1.4947, 'KRW': 1216.36, 'ZAR': 13.7908, 'CZK': 27.021, 'MXN': 20.713, 'ILS': 3.8879, 'AUD': 1.3968, 'INR': 70.496, 'RON': 4.5503, 'PHP': 53.198, 'IDR': 14144.8, 'JPY': 120.65, 'USD': 1.0556, 'HRK': 7.4213, 'PLN': 4.313, 'TRY': 3.9434, 'NZD': 1.5219, 'MYR': 4.7082, 'SEK': 9.5223, 'THB': 37.236, 'DKK': 7.4335}


In [9]:
# haluttu tulos on nyt RATES-dictionaryn avaimet aakkosjärjestyksessä tulostettuna ja pilkulla erotettuna
for country in sorted( RATES.keys() ):
    print( "{0},{1}".format( country, RATES[country] ) )
    

AUD,1.3968
BGN,1.9558
BRL,3.3151
CAD,1.4193
CHF,1.0702
CNY,7.2953
CZK,27.021
DKK,7.4335
GBP,0.86753
HKD,8.1982
HRK,7.4213
HUF,310.7
IDR,14144.8
ILS,3.8879
INR,70.496
JPY,120.65
KRW,1216.36
MXN,20.713
MYR,4.7082
NOK,8.9513
NZD,1.5219
PHP,53.198
PLN,4.313
RON,4.5503
RUB,61.7573
SEK,9.5223
SGD,1.4947
THB,37.236
TRY,3.9434
USD,1.0556
ZAR,13.7908
